In [1]:
%matplotlib inline

In [2]:
import numpy as np
import tensorflow as tf
from celeriteflow import ops

/Users/dforeman/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [8]:
session = tf.Session()

In [4]:
data = np.loadtxt('211351816.dat.ts')
t, y = np.array(data[:,0])*0.0864, np.array(data[:,1])
log_var = np.var(y)

In [5]:
T = tf.float64

t = tf.constant(np.linspace(0, 10, 500), dtype=T)
y = tf.sin(t)
diag = tf.constant(np.ones(500), dtype=T)

In [41]:
log_numax = tf.Variable(np.log(220.0), dtype=T)
log_dnu = tf.Variable(np.log(17.0), dtype=T)
gamma = tf.Variable([0.0, 0.0], dtype=T)
log_H = tf.Variable([log_var, log_var], dtype=T)
log_sigma = tf.Variable(np.log(10.0), dtype=T)
log_Q = tf.Variable(8., dtype=T)

numax = tf.exp(log_numax)
nprime = tf.range(-2.0, 3.0, dtype=T)
ell = tf.range(2, dtype=T)
nu = numax + gamma[None, :] + (tf.exp(log_dnu) * (nprime[:, None] + 0.5*ell[None, :]))
log_amp = log_H[None, :] - 0.5*tf.square(nu - numax)*tf.exp(-2*log_sigma)

S0 = tf.exp(log_amp - 2*log_Q)
w0 = 2*np.pi*nu
Q = tf.exp(log_Q)

f = tf.sqrt(4.0 * tf.square(Q) - 1.0)

nada = tf.constant([], dtype=T)
a = tf.reshape(S0 * w0 * Q, [-1])
b = tf.reshape(S0 * w0 * Q / f, [-1])
c = tf.reshape(0.5 * w0 / Q, [-1])
d = tf.reshape(0.5 * w0 / Q * f, [-1])

A, U, V, P = ops.get_celerite_matrices(nada, nada, a, b, c, d, t, diag)
D, W, _ = ops.celerite_factor(A, U, V, P)
z, _, _ = ops.celerite_solve(U, P, D, W, y[:, None])

log_like = -0.5*(tf.squeeze(tf.matmul(y[None, :], z)) + tf.reduce_sum(tf.log(D)))

session.run(tf.global_variables_initializer())

In [49]:
session.run(tf.gradients(log_like, [log_Q, log_numax, gamma]))

[64.525976004439087, 4822.0277441631679, array([ 10.8933809 ,  11.20973878])]